# This notebook is doing a random 80/20 split over all data and all sites (for the moment 25% so that not OOM) - with site-wise stratification and resampling

Load libraries and define preprocessing functions

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
import os
import xarray as xr
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
# from joblib import Parallel, delayed
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm

path = '/u/akmete/Documents/Master Thesis/Data/' #'/cluster/home/akmete/MSc/Data/'
files = [f for f in os.listdir(path) if f.endswith('.nc')]
files.sort()
assert len(files) % 3 == 0
files = {files[0 + 3 * i][:6]: (files[0 + 3 * i], files[1 + 3 * i], files[2 + 3 * i]) for i in range(len(files) // 3)}

# Function to extract good quality data
def extract_good_quality(df):
    for col in df.columns:
        if col.endswith('_qc'):
            continue
        qc_col = f"{col}_qc"
        if qc_col in df.columns:
            bad_quality_mask = df[qc_col].isin([2, 3])
            df.loc[bad_quality_mask, col] = np.nan
    return df

    # Function to initialize and clean the dataframe
def initialize_dataframe(file1, file2, file3, path, sample_fraction=0.25):
    # Open data
    ds = xr.open_dataset(path + file1, engine='netcdf4')
    dr = xr.open_dataset(path + file2, engine='netcdf4')
    dt = xr.open_dataset(path + file3, engine='netcdf4')

    # Convert to DataFrame
    df = ds.to_dataframe().reset_index()
    df_meteo = dr.to_dataframe().reset_index()
    df_rs = dt.to_dataframe().reset_index()

    # Merge dataframes on common columns
    df_combined = pd.merge(df, df_meteo, on=['time'], how='outer')
    df_combined = pd.merge(df_combined, df_rs, on=['time'], how='outer')

    # Handle longitude and latitude duplication
    if 'longitude_x' in df_combined.columns and 'latitude_x' in df_combined.columns:
        df_combined['longitude'] = df_combined['longitude_x']  # Choose longitude_x
        df_combined['latitude'] = df_combined['latitude_x']  # Choose latitude_x
        df_combined.drop(columns=['longitude_x', 'latitude_x', 'longitude_y', 'latitude_y'], inplace=True)

    # Extract and process 'DateTime' if present
    if 'time' in df_combined.columns:
        df_combined['time'] = pd.to_datetime(df_combined['time'])
        #df_combined['year'] = df_combined['time'].dt.year
        #df_combined['month'] = df_combined['time'].dt.month
        #df_combined['day'] = df_combined['time'].dt.day
        df_combined['hour'] = df_combined['time'].dt.hour
        df_combined = df_combined.drop(columns=['time'])

    # Truncate rs so that same amount of rows as flux and meteo
    df_combined = extract_good_quality(df_combined)

    # Add lagged variables, rolling mean and time features
    #df_combined['lag_1'] = df_combined['GPP'].shift(1)
    #df_combined['lag_2'] = df_combined['GPP'].shift(2)
    #df_combined['rolling_mean'] = df_combined['GPP'].rolling(window=3).mean()

    # Drop rows with NaN in the target variable
    df_combined = df_combined.dropna(subset=['GPP'])
    df_combined = df_combined.reset_index(drop=True)

    # Handle categorical variables
    string_column = 'IGBP_veg_short'
    if string_column in df_combined.columns:
        one_hot_encoded = pd.get_dummies(df_combined[string_column], prefix=string_column, dtype=int)
        df_combined = pd.concat([df_combined.drop(columns=[string_column]), one_hot_encoded], axis=1)

    # Drop quality control columns
    columns_to_drop = df_combined.filter(regex='_qc$').columns
    df_clean = df_combined.drop(columns=columns_to_drop)

    # Drop irrelevant columns
    df_clean = df_clean.drop(columns=['Qh', 'Qle', 'Qg', 'rnet', 'CO2air', 'RH', 'Qair', 'Precip', 'Psurf', 'Wind', 'NEE', 'reco', 
                                       'WWP', 'SNDPPT', 'SLTPPT', 'PHIKCL', 'PHIHOX', 'ORCDRC', 'CRFVOL', 'CLYPPT', 'CECSOL', 
                                       'BLDFIE', 'AWCtS', 'AWCh3', 'AWCh2', 'AWCh1', 'latitude', 'longitude', 'x', 'y', 'x_x', 'y_x', 'x_y', 'y_y'], errors='ignore')

    # Sample 25% of the data
    #df_clean = df_clean.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)

    return df_clean

In [20]:
sites = list(files.keys())

In [21]:
# Two dataframes, but adding additional column so that one can identify site given row of data
train_dataframe = []
test_dataframe = []
for site in tqdm(sites):
    df = initialize_dataframe(*files[site], path=path)
    df['site_id'] = site
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
    train_dataframe.append(df_train)
    test_dataframe.append(df_test)

# Stack them
train_dataframe = pd.concat(train_dataframe, ignore_index=True)
test_dataframe = pd.concat(test_dataframe, ignore_index=True)
print(train_dataframe.head())
print(test_dataframe.head())


00%|█████████████████████████████████████████| 290/290 [08:09<00:00,  1.69s/it]

            GPP    Tair     vpd   SWdown  LWdown  SWdown_clearsky  \
0 -1.165104e-08  273.55   2.746    0.000     NaN            0.000   
1  4.941566e-09  293.85   6.519    0.000     NaN            0.000   
2  6.272396e-08  286.25   8.904  110.623     NaN          469.518   
3 -8.584021e-10  294.65  16.372    0.000     NaN            0.000   
4 -2.998726e-08  268.95   4.386    0.000     NaN            0.000   

   LST_TERRA_Day  LST_TERRA_Night       EVI      NIRv  ...  \
0         283.42           272.94  0.166064  0.047277  ...   
1         317.56           293.96  0.226728  0.071960  ...   
2         291.16           276.28  0.183343  0.054505  ...   
3         297.17           285.44  0.170636  0.048492  ...   
4         286.82           274.20  0.183200  0.054450  ...   

   IGBP_veg_short_b'ENF'  IGBP_veg_short_b'SAV'  IGBP_veg_short_b'EBF'  \
0                    NaN                    NaN                    NaN   
1                    NaN                    NaN                 

In [22]:
# Define a function which resamples/undersamples so that we get the same amount of data from every site
def resample_sites(df, site_col='site_id', min_samples=20000, max_samples=25000, random_state=42):
    df_list = []
    for site in df[site_col].unique():
        site_data = df[df[site_col]==site]
        site_len = len(site_data)

        # Undersample if above max
        if site_len > max_samples:
            site_data = site_data.sample(n=max_samples, random_state=random_state)

        # Oversample if below min
        elif site_len < min_samples:
            site_data = site_data.sample(n=min_samples, replace=True, random_state=random_state)

        df_list.append(site_data)

    df_balanced = pd.concat(df_list, axis=0)
    return df_balanced

In [23]:
# Resample in Train but leave Test untouched
train_dataframe_balanced = resample_sites(df=train_dataframe, site_col='site_id', min_samples=20000, max_samples=25000, random_state=42)

# Define feature and target variables
X_train = train_dataframe_balanced.drop(columns=['GPP', 'site_id'])
y_train = train_dataframe_balanced['GPP']

X_test = test_dataframe.drop(columns=['GPP', 'site_id'])
y_test = test_dataframe['GPP']

In [24]:
# Scale y (minmax scaling)
y_train_standard = (y_train - y_train.mean()) / (y_train.max() - y_train.min())
y_test_standard = (y_test - y_train.mean()) / (y_train.max() - y_train.min())

# Scale X (minmax scaling)
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform X_train
X_train_scaled = scaler.fit_transform(X_train)

# Transform X_test using the same scaler
X_test_scaled = scaler.transform(X_test)

In [25]:
# Set up model
model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)

# Train model
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [26]:
# Make predictions and calculate metrics
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test_standard, y_pred)
r2 = r2_score(y_test_standard, y_pred)
relative_error = np.mean(np.abs(y_test_standard - y_pred) / np.abs(y_test_standard))
mae = np.mean(np.abs(y_test_standard - y_pred))
rmse = np.sqrt(mse)
print(mse)
print(r2)
print(rmse)
print(mae)
print('done')

0.00115526104831901
-0.00044479788689288924
0.03398913132633739
0.023058014620932564
